In [4]:
# 4-2

# =========================================================
# BERT（Bidirectional Encoder Representations from Transformers）の理論メモ
# ---------------------------------------------------------
# ■ 目的
#  - 文脈の双方向（左←→右）を同時に考慮した潜在表現 h ∈ R^H を得る。
#  - 事前学習（Pretraining）で一般的言語能力を獲得し、下流タスク（分類・系列ラベリング・QA等）に微調整（Fine-tuning）で適用。
#
# ■ 事前学習タスク
#  - MLM（Masked Language Modeling）:
#    入力トークン列 x の一部を [MASK] 等で隠し、p(x_masked | context) を最大化する。
#    近似的には、各マスク位置 i に対して softmax(W h_i) の対数尤度を最大化。
#  - NSP（Next Sentence Prediction）:
#    文 A の直後に文 B が来るかを 2 値分類（Tohoku BERT では採用）。RoBERTa 系では廃止例もある。
#
# ■ モデル（BertModel）の内部
#  - 入力埋め込み: token + position + segment（token_type）を加算して X ∈ R^{B×L×H} を作る。
#  - Transformer Encoder（L層）: 各層で Multi-Head Self-Attention + FFN。
#    • Attention は Scaled Dot-Product:
#        Q = XW_Q, K = XW_K, V = XW_V  (各 W_* ∈ R^{H×d_k})
#        scores = Q K^T / sqrt(d_k)
#        A = softmax(scores + mask)
#        head = A V
#      複数 head を結合し線形写像で次層へ。
#    • 計算量: O(L^2 H)（長文入力ほど二乗で増える）; メモリもほぼ O(L^2)。
#  - 出力:
#    • last_hidden_state: 形状 [B, L, H]。各トークン位置の文脈化表現。
#    • pooler_output: 形状 [B, H]。最終層の [CLS] ベクトルに線形＋tanh を適用（主に文分類の初期ベースライン）。
#    • hidden_states, attentions（オプション）: 各層の中間表現やアテンション行列を返す。
#
# ■ 日本語トークナイザ（BertJapaneseTokenizer）
#  - 典型的には形態素解析（MeCab/fugashi 等）で語彙単位に分割 → WordPiece に細分化。
#  - 特殊トークン: [CLS]（先頭）, [SEP]（区切り）, [PAD], [MASK], [UNK] など。
#  - 下流入力の慣例:
#      input_ids        : [B, L]（語彙ID）
#      attention_mask   : [B, L]（pad 0/非pad 1）
#      token_type_ids   : [B, L]（文 A=0 / 文 B=1）
#    これらを model(**batch) で渡す。
#
# ■ 実運用の注意
#  - 最大系列長: 多くの BERT は L ≤ 512。超える場合は分割や Longformer 等の検討。
#  - 学習: AdamW + 学習率ウォームアップ + 余弦減衰などが定番。weight decay は LayerNorm/バイアス除外。
#  - 乱数・再現性: torch.manual_seed, numpy.random.seed を固定。dropout の影響も考慮。
#  - 参照: Devlin et al., 2018; Tohoku NLP "cl-tohoku/bert-base-japanese" 系列
# =========================================================

import torch
from transformers import BertJapaneseTokenizer, BertModel

# ---------------------------------------------------------
# 依存関係の注記（インストール時の目安）
#  - transformers >= 4.x
#  - fugashi, ipadic（や unidic-lite）: BertJapaneseTokenizer の形態素解析で利用されることが多い
#    例) pip install transformers fugashi ipadic
# ---------------------------------------------------------

# ---------------------------------------------------------
# （参考）実行例：事前学習済み日本語BERTの読み込み
#   ※ 下の例は「使い方の目安」を示すためのコメントであり、実行は任意です。
#
# # 1) 日本語BERTのトークナイザ／モデルをロード
# tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
# model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")
# model.eval()  # 推論時は eval モードで dropout 等を停止
#
# # 2) テキストを ID 列へ変換
# #   return_tensors="pt" で PyTorch テンソルを自動作成
# batch = tokenizer(
#     ["今日は良い天気ですね。", "しかし少し風が強いかもしれません。"],
#     padding=True, truncation=True, max_length=128, return_tensors="pt"
# )
# # batch: dict で input_ids/attention_mask/token_type_ids を含む
# # 形状: input_ids.shape == [B, L]
#
# # 3) モデルへ前向き計算
# with torch.no_grad():
#     outputs = model(**batch, output_attentions=False, output_hidden_states=False)
#
# # 4) 出力の形状
# # outputs.last_hidden_state: [B, L, H]  各トークン位置の表現
# # outputs.pooler_output   : [B, H]      [CLS] 由来（文表現のベースライン）
#
# # 5) 用途の例
# # - 文分類: pooler_output か、[CLS] の last_hidden_state[:, 0, :] を線形分類器へ
# # - トークン分類（固有表現抽出など）: last_hidden_state を各位置で線形＋softmax
# # - QA: [CLS]/[SEP] を含むペア入力で start/end スパンを回帰
#
# # 6) 理論と設計の対応
# # - attention_mask は pad 位置へのソフトマックスを -∞ マスクで抑制（scores に加算）
# # - token_type_ids は NSP/ペア入力の区別（A:0, B:1）。単文なら全 0 でよい。
# # - 学習では MLM/NSP（または SOP）に対応するヘッドを別途持つ（BertForPreTraining 等）。
# ---------------------------------------------------------

In [5]:
# 4-3

# ---------------------------------------------------------
# 理論メモ：Whole Word Masking（WWM）版日本語BERTを使う理由
# ---------------------------------------------------------
# ◇ 標準MLM vs. Whole Word Masking
#  - 標準のMLM（WordPiece単位の隠し方）では、語が「サブワード」に分割された場合、
#    その一部のサブワードだけが[MASK]になることが多い。
#  - WWMは「同じ語（= 形態素→WordPieceに分解される前の“語”）」に属する全サブワードを
#    まとめて同時にマスクする。これによりモデルは「語レベルの完形復元」を学ぶ圧力が強まり、
#    文脈一貫性や語彙レベルの表現獲得に寄与する（サブワード断片の当て推量を減らす）。
#  - 直感的には、より強い隠し課題（harder pretext task）→汎化に寄与する可能性。
#    ただし学習難易度は上がりうる（学習安定化はデータとハイパラ次第）。
#
# ◇ 日本語での影響
#  - 日本語は語境界が空白で明示されないため、形態素解析（例：MeCab + 辞書）で語単位を推定→
#    さらにWordPieceで細分化、という段階を踏む。この「語」単位でマスクを丸ごと掛けるのがWWM。
#  - 形態素解析の精度や辞書選択（ipadic / unidic-lite 等）に依存する側面がある。
#
# ◇ 実務上の注意
#  - このモデル名 'tohoku-nlp/bert-base-japanese-whole-word-masking' は、
#    東北大NLPが公開している日本語BERT（WWM版）の重み・語彙を指す。
#  - Tokenizer/Modelは対応するペアを使う（語彙の不一致はIDずれ→性能劣化）。
#  - 依存関係：transformers、fugashi、ipadic（またはunidic-lite）等を事前にインストール。
#    例：pip install transformers fugashi ipadic
# ---------------------------------------------------------

model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"  # WWM版日本語BERT（語レベルでの一括マスキングで事前学習）
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# （参考）挙動確認スニペット：※必要なら実行
# text = "深層学習の事前学習モデルBERTは強力です。"
# enc = tokenizer(
#     text,
#     return_tensors="pt",      # PyTorchテンソルで返す
#     padding="max_length",     # 最大長にパディング（推論/バッチ化の整列用）
#     truncation=True,          # 上限長超過時に切り詰め
#     max_length=32
# )
# # enc["input_ids"].shape == [1, 32], enc["attention_mask"].shape == [1, 32]
# # WWMは事前学習時のマスキング戦略であり、推論時のトークナイズ結果（input_ids自体）が
# # 直接変わるわけではない点に注意（学習済み表現に差が出る）。
#
# # なお、下流タスクでは対応するモデル（例：BertModel/BertForSequenceClassification など）を
# # 同じ `model_name` から読み出し、tokenizerとペアで使うことが重要。

tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [6]:
# 4-4 ～ 4-6
# =========================================================
# 理論補足：BertJapaneseTokenizerのトークナイズ動作（日本語×WordPiece）
# ---------------------------------------------------------
# ■ 概要
#  - BertJapaneseTokenizer は「形態素解析 → WordPiece 分割」という2段階でトークン化する。
#    1) 形態素解析：文を語（形態素）へ分割（MeCab + 辞書 ipadic / unidic-lite 等）
#    2) WordPiece：各「語」を語彙に基づいて更にサブワードへ分割（語彙外は細分化、最悪 [UNK]）
#  - 返り値 tokenizer.tokenize(text) は特殊トークンを付与しない素のサブワード列（list[str]）。
#    例：['自', '然', '言', '語', '処', '理'] のような字単位、あるいは ['自然', '言語', '処理'] など
#    語彙と辞書の組み合わせにより変動する（＝「必ずこの分割になる」とは限らない）。
#
# ■ Whole Word Masking（WWM）との関係
#  - 本トークナイザ自体は「推論・前処理」でのサブワード列を返すのみ。
#  - WWM は「事前学習時のマスク戦略」で、同一“語”（形態素に相当）に属するサブワードを
#    まとめて同時にマスクする。推論時の tokenize 結果は WWM の有無で直接は変わらない。
#
# ■ 文ごとの観察ポイント
#  (A) 「明日は自然言語処理の勉強をしよう。」
#    - 「自然言語処理」は複合名詞。形態素解析の辞書設定により
#      「自然/言語/処理」や「自然言語/処理」等に分かれうる。
#    - その後、WordPiece がさらに細分化（語彙にあればそのまま、無ければ細切れ）。
#  (B) 「明日はマシンラーニングの勉強をしよう。」
#    - カタカナ複合語。「マシン」「ラーニング」に分かれる場合や
#      「マシンラーニング」単語→サブワード細分の可能性がある。
#  (C) 「機械学習を中国語にすると机器学习だ。」
#    - 末尾の「机器学习」は中国語（簡体字）。日本語辞書では未知語扱いになりやすく、
#      WordPiece が文字単位や部分列へ分解、最悪 [UNK] へフォールバックしうる。
#    - BERT の日本語語彙は CJK を広く含むため [UNK] にならず字単位分割で拾えることも多い。
#
# ■ 実務上のTips
#  - 再現性：辞書種別（ipadic / unidic-lite）・バージョン、transformers のバージョンで分割が変わる。
#  - 下流処理：tokenize → convert_tokens_to_ids → [CLS]/[SEP] 付与 → attention_mask 作成、が一般手順。
#  - 形状把握：下流モデル入力は通常 encode_plus / __call__（return_tensors="pt"）を使うのが安全。
# =========================================================

# 既に 4-3 で `tokenizer` を作っている想定。
# セッション直実行でも動くように保険として用意（未定義なら初期化）。
try:
    tokenizer  # 変数が存在するかチェック
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# 4-4：語彙・辞書により複合名詞「自然言語処理」の扱いが変動しうる点に注目
tokens_44 = tokenizer.tokenize("明日は自然言語処理の勉強をしよう。")
# デバッグ表示（必要ならコメント解除）
# print(tokens_44)

# 4-5：カタカナ複合語の分割（「マシン」「ラーニング」等）やサブワード接頭辞（##）の有無に注目
tokens_45 = tokenizer.tokenize("明日はマシンラーニングの勉強をしよう。")
# print(tokens_45)

# 4-6：簡体字中国語「机器学习」の未知語挙動（字単位分割や [UNK] フォールバック）に注目
tokens_46 = tokenizer.tokenize("機械学習を中国語にすると机器学习だ。")
# print(tokens_46)

# （参考）ID 列へ変換して下流モデル入力へ繋ぐ例：
# enc_44 = tokenizer('明日は自然言語処理の勉強をしよう。', return_tensors='pt')
# # enc_44 は input_ids / attention_mask / token_type_ids を含む辞書
# # enc_44['input_ids'].shape == [1, L]

In [7]:
# 4-7
# =========================================================
# 理論メモ：`tokenize` と `encode` の違い（BERT日本語，WWM版）
# ---------------------------------------------------------
# ■ `tokenize(text)`：サブワード列（strのリスト）を返すだけ。特殊トークンは付与しない。
# ■ `encode(text)`  ：語彙ID列（intのリスト）を返す。既定で [CLS] と [SEP] が付与される
#                      （transformers では `add_special_tokens=True` が既定）。
# つまり、学習/推論でモデルに渡す最小単位は「ID列」であり、`encode` または
# `tokenizer(..., return_tensors='pt')` を使ってテンソル化するのが実務的。
#
# ■ 特殊トークン
#  - [CLS]：文頭（文分類などで表現を使用）
#  - [SEP]：文区切り（単文でも文末に付く）
#  - [PAD]：バッチ整列用（`padding` で追加）
#  - [MASK]：事前学習MLMで使用（推論の前処理では出てこない）
#
# ■ 再現性の注意
#  - 形態素辞書（ipadic / unidic-liteなど）と語彙（model_name）に依存してIDが決まる。
#  - 同じテキストでも tokenizer/model の種類やバージョンが異なると ID は変わりうる。
#
# ■ 下流タスクへの橋渡し
#  - `encode` は ID のみで attention_mask を返さない（= PAD無視マスクが無い）。
#    実運用は `tokenizer(text, return_tensors='pt', padding=..., truncation=...)`
#    で `input_ids / attention_mask / token_type_ids` を一括生成するのが安全。
# =========================================================

# 依存：4-3 で tokenizer が定義済みの想定だが、未定義なら初期化しておく
try:
    tokenizer
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# --- 観察対象テキスト ---
text = "明日は自然言語処理の勉強をしよう。"

# 1) encode：ID列（既定で [CLS], [SEP] 付き）を得る
input_ids = tokenizer.encode(text)  # add_special_tokens=True（既定）
print(">> input_ids:", input_ids)

# 2) 可読化：ID → サブワード列（特殊トークンを含む）
tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(">> tokens:", tokens)
#   実務メモ：先頭が [CLS]、末尾が [SEP] になっていることを確認すると下流処理が安定

# 3) 逆変換：ID列 → 文字列（decode）
decoded_with_special = tokenizer.decode(input_ids, skip_special_tokens=False)
decoded_without_special = tokenizer.decode(input_ids, skip_special_tokens=True)
print(">> decode (with special):", decoded_with_special)
print(">> decode (no  special):", decoded_without_special)
#   理論的含意：decodeは可逆でない場合がある（空白や正規化の扱い，未登録語の分解など）

# 4) 実運用：モデル入力で必要なテンソルを一括生成（推奨パス）
batch = tokenizer(
    text,
    return_tensors="pt",  # PyTorchテンソル化
    padding=False,  # 単文なので不要（バッチなら True/ "longest"/ "max_length" など）
    truncation=True,  # 最大長越えの安全弁（デフォルトでは切り捨てない）
    max_length=128,
)
print(">> batch keys:", list(batch.keys()))
print(
    ">> shapes:",
    "input_ids",
    tuple(batch["input_ids"].shape),
    "attention_mask",
    tuple(batch["attention_mask"].shape),
    "token_type_ids",
    tuple(batch["token_type_ids"].shape),
)
#   理論メモ：attention_mask は softmax(QK^T/√d_k) の前に「PAD位置へ -∞ 加算」相当の抑制に使われる
#             （= PADに注意が向かない）。token_type_ids は文A=0/文B=1 の区別（単文ならゼロ列）。

# 5) 参考：長文での切り詰め・パディング（計算量 O(L^2) への配慮）
long_text = text * 100
enc_packed = tokenizer(
    long_text,
    return_tensors="pt",
    padding="max_length",  # 事前に固定長へ整列（デプロイでのスループット向上）
    truncation=True,  # 最大長を超えた分をカット
    max_length=64,  # 例：64に固定（BERT標準は最大512）
)
print(
    ">> packed shapes:",
    "input_ids",
    tuple(enc_packed["input_ids"].shape),
    "attention_mask",
    tuple(enc_packed["attention_mask"].shape),
)
print(">> non-pad length:", int(enc_packed["attention_mask"].sum().item()))
#   理論メモ：Self-Attention の計算量は O(L^2 H)。L（系列長）を管理する設計は実運用で重要。

>> input_ids: [2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 28489, 8, 3]
>> tokens: ['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', '##う', '。', '[SEP]']
>> decode (with special): [CLS] 明日 は 自然 言語 処理 の 勉強 を しよう 。 [SEP]
>> decode (no  special): 明日 は 自然 言語 処理 の 勉強 を しよう 。
>> batch keys: ['input_ids', 'token_type_ids', 'attention_mask']
>> shapes: input_ids (1, 13) attention_mask (1, 13) token_type_ids (1, 13)
>> packed shapes: input_ids (1, 64) attention_mask (1, 64)
>> non-pad length: 64


In [8]:
# 4-8
# =========================================================
# 理論メモ：`convert_ids_to_tokens` の役割と `decode` との違い
# ---------------------------------------------------------
# ■ 役割
#   - `convert_ids_to_tokens(ids)` は、各 ID を **WordPiece サブワード** 文字列へ 1:1 で写像する。
#   - 特殊トークン（[CLS], [SEP], [PAD], [MASK], [UNK]）も **そのまま文字列**として返す。
#
# ■ `decode` との違い
#   - `decode(ids)` は **可読な文章**へ連結・正規化を行う（特殊トークンは既定で除去可能）。
#   - 一方 `convert_ids_to_tokens` は **分割境界を保持**（学習・解析・デバッグ向け）。
#
# ■ 実務的含意
#   - NER 等のトークン分類では **サブワード境界**が重要なので `convert_ids_to_tokens` で形を確認。
#   - 文章表示は `decode` / `convert_tokens_to_string` を使う（可逆でない場合あり：空白・正規化・未知語）。
#   - WWM（Whole Word Masking）は「事前学習時のマスク戦略」であり、ここでの表示自体は変えない。
# =========================================================

# 依存：4-3/4-7 で tokenizer と input_ids が定義済みの想定。なければ初期化して作る。
try:
    tokenizer  # 存在確認
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

try:
    input_ids  # 存在確認
except NameError:
    # 4-7 相当の最小再現
    text = "明日は自然言語処理の勉強をしよう。"
    input_ids = tokenizer.encode(text)  # 既定で [CLS], [SEP] が付与される

# 1) ID → サブワード列（特殊トークン含む）
tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(">> tokens:", tokens)

# 2) 可逆性の軽い確認：tokens → ids → tokens
roundtrip_ids = tokenizer.convert_tokens_to_ids(tokens)
roundtrip_tokens = tokenizer.convert_ids_to_tokens(roundtrip_ids)
print(">> roundtrip equal (ids):", roundtrip_ids == input_ids)
print(">> roundtrip equal (toks):", roundtrip_tokens == tokens)

# 3) 文章可読化パス（分割境界を潰して連結）
text_from_tokens = tokenizer.convert_tokens_to_string(tokens)
text_from_decode_keep = tokenizer.decode(
    input_ids, skip_special_tokens=False
)  # 特殊トークン表示
text_from_decode_skip = tokenizer.decode(
    input_ids, skip_special_tokens=True
)  # 特殊トークン除去
print(">> convert_tokens_to_string:", text_from_tokens)
print(">> decode (keep specials):  ", text_from_decode_keep)
print(">> decode (skip  specials): ", text_from_decode_skip)

# 4) 解析のコツ：特殊トークンや未知語の存在確認
specials = {"[CLS]", "[SEP]", "[PAD]", "[MASK]", "[UNK]"}
has_unk = any(t == "[UNK]" for t in tokens)
present_specials = sorted(set(t for t in tokens if t in specials))
print(">> specials in tokens:", present_specials)
print(">> contains [UNK]    :", has_unk)

# 5) （参考）特殊トークンを除いた実トークン列
tokens_wo_specials = [t for t in tokens if t not in specials]
print(">> tokens (no specials):", tokens_wo_specials)

# 6) 補足理論メモ：
#   - WordPiece は未知語を細分化し、語彙に存在しない断片は最終的に [UNK] へ落ちる場合がある。
#   - 日本語では「形態素解析→WordPiece」の二段で分割されるため、辞書・語彙・バージョンで結果が揺れる。
#   - サブワード接頭辞（例：'##学' のような継続マーク）は語彙に依存（日本語BERTでも用いられる）。
#   - トークン分類では「語→サブワード」のアライメント管理が必要（fast系トークナイザなら offset_mapping）。

>> tokens: ['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', '##う', '。', '[SEP]']
>> roundtrip equal (ids): True
>> roundtrip equal (toks): True
>> convert_tokens_to_string: [CLS] 明日 は 自然 言語 処理 の 勉強 を しよう 。 [SEP]
>> decode (keep specials):   [CLS] 明日 は 自然 言語 処理 の 勉強 を しよう 。 [SEP]
>> decode (skip  specials):  明日 は 自然 言語 処理 の 勉強 を しよう 。
>> specials in tokens: ['[CLS]', '[SEP]']
>> contains [UNK]    : False
>> tokens (no specials): ['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', '##う', '。']


In [9]:
# 4-9
# =========================================================
# 理論メモ：パディング（padding='max_length'）と切り詰め（truncation=True）の意味
# ---------------------------------------------------------
# ■ 何が起きるか
#  - `max_length=12`：出力系列長 L を 12 に固定する（特殊トークン [CLS]/[SEP] を含む）。
#  - `padding='max_length'`：不足ぶんを [PAD] で右詰めパディングし、attention_mask は PAD 位置を 0 に。
#  - `truncation=True`：L を超える場合、右側を切り詰める（通常 [CLS] 先頭固定、[SEP] は末尾側に残す）。
#
# ■ Attention とマスク
#  - Self-Attention は softmax(QK^T / √d_k) V。PAD 位置は mask によりスコアへ -∞（相当）を加算し無視。
#  - よって attention_mask=0 のトークン（PAD）には注意が向かず、勾配も基本的に流れない。
#
# ■ token_type_ids
#  - 単文入力では通常すべて 0。文対（A,B）では A=0, B=1 で区別（NSP系やQAで利用）。
#
# ■ 実務含意
#  - バッチ推論/学習で長さをそろえるための固定長化（throughput・安定化）。
#  - 計算量 O(L^2 H) のため、L を管理する設計は重要（過大な max_length は計算/メモリ負荷）。
# =========================================================

# 依存：前セルで tokenizer が定義済みの前提（なければ初期化）
try:
    tokenizer
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# --- 入力 ---
text = "明日の天気は晴れだ。"

# --- エンコード：固定長（L=12）でのパディング・切り詰め ---
encoding = tokenizer(text, max_length=12, padding="max_length", truncation=True)
print("# encoding (BatchEncoding repr):")
print(encoding)  # dictライクな表示（input_ids / attention_mask / token_type_ids）

# 個別に中身を明示（学習・デバッグ時に便利）
print("\n# input_ids:", encoding["input_ids"])
print("# attention_mask:", encoding["attention_mask"])
print("# token_type_ids:", encoding.get("token_type_ids"))

# --- 可視化：ID -> サブワード列（特殊トークン込み） ---
tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
print("\n# tokens:")
print(tokens)

# --- 解析：特殊トークンの位置と有効長（非PAD長） ---
CLS_ID = tokenizer.cls_token_id
SEP_ID = tokenizer.sep_token_id
PAD_ID = tokenizer.pad_token_id

ids = encoding["input_ids"]
mask = encoding["attention_mask"]

cls_pos = ids.index(CLS_ID) if CLS_ID in ids else None
sep_pos = ids.index(SEP_ID) if SEP_ID in ids else None
pad_positions = [i for i, t in enumerate(ids) if t == PAD_ID]
effective_len = sum(mask)  # 非PAD長 = 有効トークン数

print("\n# specials & length:")
print("CLS at:", cls_pos, " / SEP at:", sep_pos, " / PAD positions:", pad_positions)
print("effective (non-PAD) length =", effective_len, " / total length =", len(ids))

# --- デコード（人が読む文字列へ） ---
decoded_keep = tokenizer.decode(ids, skip_special_tokens=False)
decoded_skip = tokenizer.decode(ids, skip_special_tokens=True)
print("\n# decode (with specials):", decoded_keep)
print("# decode (skip specials): ", decoded_skip)

# --- 切り詰めの挙動確認（長文にして強制的にトランケーション） ---
long_text = text * 20  # わざと長く
enc_trunc = tokenizer(
    long_text,
    max_length=12,
    padding="max_length",
    truncation=True,
    return_overflowing_tokens=True,  # どれだけ削られたかのメタ情報を得る
)
print("\n# truncation check:")
print("input_ids (truncated)  :", enc_trunc["input_ids"])
print("attention_mask         :", enc_trunc["attention_mask"])
print("num_truncated_tokens   :", enc_trunc.get("num_truncated_tokens"))
print("overflowing_tokens (len):", len(enc_trunc.get("overflowing_tokens", [])))
print(
    "tokens (truncated)     :", tokenizer.convert_ids_to_tokens(enc_trunc["input_ids"])
)

# 理論補足：
# - 右側切り詰めでは [CLS] は index 0 に残り、[SEP] は系列の末端側（通常 index = 有効長-1）を占める。
# - 残りは [PAD] で埋められ、attention_mask=0 となるため、Self-Attention の計算から実質除外される。
# - 文字列→サブワード→ID の過程で完全可逆性は保証されない（空白・正規化・未知語などの影響）。

# encoding (BatchEncoding repr):
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}

# input_ids: [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0, 0]
# attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
# token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]']

# specials & length:
CLS at: 0  / SEP at: 8  / PAD positions: [9, 10, 11]
effective (non-PAD) length = 9  / total length = 12

# decode (with specials): [CLS] 明日 の 天気 は 晴れ だ 。 [SEP] [PAD] [PAD] [PAD]
# decode (skip specials):  明日 の 天気 は 晴れ だ 。

# truncation check:
input_ids (truncated)  : [2, 11475, 5, 11385, 9, 16577, 75, 8, 11475, 5, 11385, 3]
attention_mask         : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
num_truncated_tokens   : 130
overflowing_tokens (len): 130
tokens (truncated)     : ['[CLS]', '明日', 'の', '天気',

In [10]:
# 4-10
# =========================================================
# 理論メモ：極小 max_length（=6）でのトランケーションとパディング
# ---------------------------------------------------------
# ■ 目標
#   - 出力系列長 L を 6 に固定しつつ、[CLS]/[SEP] を必ず含める。
#   - 単文入力のため、最大で「L-2=4 個」のサブワードしか保持できない。
#
# ■ 何が起こるか（単文・右詰め想定）
#   - 変換順：text → 形態素解析 → WordPiece → 特殊トークン付与 → 切り詰め → パディング
#   - 切り詰め（truncation=True）：右側（末尾側）から削る（単文は first sequence のみを削減）。
#   - パディング（padding='max_length'）：不足分を [PAD] で右詰め。attention_mask=0 で無視。
#   - 結果：index=0 に [CLS]、index = 有効長-1 に [SEP]、末尾側は [PAD] が埋まる（またはトークンが削られる）。
#
# ■ Self-Attention への影響
#   - softmax(QK^T/√d_k) において、PAD 位置は mask によりスコアへ -∞ 相当が加わり、注意が向かない。
#   - 有効長（non-PAD 長）を最小限にすると、計算量 O(L^2 H) は抑えられるが、文脈の損失（情報欠落）と交換になる。
#
# ■ 再現性の注意
#   - 「保持できるサブワード数」は固定（ここでは 4）だが、実際にどのサブワードがそこに入るかは
#     形態素辞書・語彙・バージョンに依存（= tokenize 結果が揺れる）。ID列の完全一致を要件にしない。
# =========================================================

# 依存：前セルで tokenizer と text が定義済みの前提。未定義なら保険として初期化する。
try:
    tokenizer  # 存在確認
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

try:
    text  # 存在確認（前セル #4-9 と同一を想定）
except NameError:
    text = "明日の天気は晴れだ。"

# --- エンコード：L=6 で固定長、右側切り詰め＋右詰めPad ---
encoding = tokenizer(text, max_length=6, padding="max_length", truncation=True)

# --- 表示：サブワード列（特殊トークン込み） ---
tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
print(tokens)

# --- 解析（参考）：有効長と特殊トークンの配置を確認 ---
ids = encoding["input_ids"]
mask = encoding["attention_mask"]

CLS_ID = tokenizer.cls_token_id
SEP_ID = tokenizer.sep_token_id
PAD_ID = tokenizer.pad_token_id

cls_pos = ids.index(CLS_ID) if CLS_ID in ids else None
sep_pos = ids.index(SEP_ID) if SEP_ID in ids else None
pad_positions = [i for i, t in enumerate(ids) if t == PAD_ID]
effective_len = sum(mask)  # 非PAD長

print("# len(ids)=", len(ids), " effective(non-PAD)=", effective_len)
print("# CLS at", cls_pos, "/ SEP at", sep_pos, "/ PAD positions", pad_positions)

# --- どれだけ削られた可能性があるか（理論キャパ：L-2=4 サブワード） ---
# tokenize（特殊トークンなし）で生のサブワード長を観察し、理論上の削減量を見積もる。
raw_tokens = tokenizer.tokenize(text)
capacity = 6 - 2  # [CLS] と [SEP] のぶん
truncated_count_est = max(0, len(raw_tokens) - capacity)
print(
    "# raw subwords =",
    len(raw_tokens),
    " capacity =",
    capacity,
    " -> truncated(est)=",
    truncated_count_est,
)

# --- 人が読む表示（可逆ではない点に注意） ---
decoded_keep = tokenizer.decode(ids, skip_special_tokens=False)
decoded_skip = tokenizer.decode(ids, skip_special_tokens=True)
print("# decode(with specials):", decoded_keep)
print("# decode(skip  specials):", decoded_skip)

# 補足：
# - 「切り詰め優先度」は単文では first sequence の右側。ペア入力では longest_first が既定（必要に応じ設定可）。
# - L を極端に小さくすると [CLS]/[SEP] を除く保持域が狭く、文脈情報が失われやすい（下流性能に影響）。
# - 固定長化はスループット安定化に有用だが、現実の文長分布を踏まえた L 設計（あるいは動的パディング）が望ましい。

['[CLS]', '明日', 'の', '天気', 'は', '[SEP]']
# len(ids)= 6  effective(non-PAD)= 6
# CLS at 0 / SEP at 5 / PAD positions []
# raw subwords = 7  capacity = 4  -> truncated(est)= 3
# decode(with specials): [CLS] 明日 の 天気 は [SEP]
# decode(skip  specials): 明日 の 天気 は


In [11]:
# 4-11
# =========================================================
# 理論メモ：ミニバッチ入力（list[str]）＋固定長エンコードの挙動
# ---------------------------------------------------------
# ■ 目的
#  - 2文を同時にトークナイズし、系列長 L=10 に固定（padding='max_length'）する。
#  - 各文に [CLS]・[SEP] を必ず含めるので、実トークンの保持域は L-2=8 サブワードまで。
#
# ■ 生成される3要素
#  - input_ids      : 各サブワード（含む特殊トークン）を語彙IDへ写像した配列（B×L）
#  - attention_mask : 非PAD=1, PAD=0（Self-Attention の softmax 前スコアに -∞ 相当を与え注意抑制）
#  - token_type_ids : 単文は全0（文対のとき A=0, B=1）
#
# ■ トランケーション（truncation=True）
#  - 単文バッチなので「右側を優先的に切り詰め（only_first 相当）」→ 末尾側のサブワードが削られる。
#  - [CLS] は index=0 に、[SEP] は「有効長-1」に配置されるのが不変量。
#
# ■ 実務上の含意
#  - 長さ L の管理は計算量 O(L^2 H) に効く（BERTのSelf-Attentionは系列長二乗）。
#  - 完全一致のトークン列は「辞書・語彙・バージョン」に依存して揺れるため、再現性要件は“性質”で置く。
#    （例：有効長が上限内、特殊トークンの位置、PADのマスク整合 など）
# =========================================================

# 保険：前セルで tokenizer が未定義なら初期化
try:
    tokenizer
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# --- 入力バッチ（B=2） ---
text_list = [
    "明日の天気は晴れだ。",
    "パソコンが急に動かなくなった。",
]  # 例：単文2件（文対ではない）

# --- エンコード：固定長 L=10、右側切り詰め＋右詰めPad ---
encoding = tokenizer(text_list, max_length=10, padding="max_length", truncation=True)

# 表示：BatchEncoding は dict ライク。学習・デバッグで中身を明示する。
print("# keys:", list(encoding.keys()))
print("# input_ids[0]:", encoding["input_ids"][0])
print("# input_ids[1]:", encoding["input_ids"][1])
print("# attention_mask[0]:", encoding["attention_mask"][0])
print("# attention_mask[1]:", encoding["attention_mask"][1])
print("# token_type_ids[0]:", encoding["token_type_ids"][0])
print("# token_type_ids[1]:", encoding["token_type_ids"][1])

# --- 可視化：ID → サブワード列（特殊トークン込み） ---
for i, ids in enumerate(encoding["input_ids"]):
    toks = tokenizer.convert_ids_to_tokens(ids)
    print(f"\n# sample[{i}] tokens:")
    print(toks)

    # 不変量チェック：特殊トークン位置と有効長
    CLS_ID = tokenizer.cls_token_id
    SEP_ID = tokenizer.sep_token_id
    PAD_ID = tokenizer.pad_token_id

    mask = encoding["attention_mask"][i]
    cls_pos = ids.index(CLS_ID) if CLS_ID in ids else None
    sep_pos = ids.index(SEP_ID) if SEP_ID in ids else None
    pad_positions = [j for j, t in enumerate(ids) if t == PAD_ID]
    effective_len = sum(mask)  # 非PAD長

    print("# specials & length:")
    print("  CLS at:", cls_pos, "/ SEP at:", sep_pos, "/ PAD positions:", pad_positions)
    print(
        "  effective (non-PAD) length =", effective_len, " / total length =", len(ids)
    )

    # 参考：人が読む表示（可逆ではない点に注意）
    print("# decode (skip specials):", tokenizer.decode(ids, skip_special_tokens=True))

# --- どれだけ削られた可能性があるか：理論キャパ（L-2）と比較 ---
capacity = 10 - 2  # [CLS] と [SEP] のぶん
for i, text in enumerate(text_list):
    raw = tokenizer.tokenize(text)  # 特殊トークン無しのサブワード列
    truncated_est = max(0, len(raw) - capacity)
    print(
        f"\n# sample[{i}] raw_subwords={len(raw)} capacity={capacity} -> truncated(est)={truncated_est}"
    )
    # 備考：実際の削減は特殊トークン付与後・右側優先。語彙・辞書差で raw の値は変動しうる。

# --- 参考：実運用（学習/推論）ではテンソル出力を一括生成するのが安全 ---
batch_pt = tokenizer(
    text_list,
    max_length=10,
    padding="max_length",
    truncation=True,
    return_tensors="pt",  # ← PyTorch テンソルで返す（B×L）
)
print(
    "\n# tensor shapes:",
    "input_ids",
    tuple(batch_pt["input_ids"].shape),
    "attention_mask",
    tuple(batch_pt["attention_mask"].shape),
    "token_type_ids",
    tuple(batch_pt["token_type_ids"].shape),
)
# 理論補足：attention_mask=0 の位置は Self-Attention の softmax 前に -∞ 相当が加算され、注意が向かない。
#           これにより PAD トークンは表現学習に寄与せず、勾配も基本的に流れない。

# keys: ['input_ids', 'token_type_ids', 'attention_mask']
# input_ids[0]: [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0]
# input_ids[1]: [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]
# attention_mask[0]: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
# attention_mask[1]: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# token_type_ids[0]: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# token_type_ids[1]: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# sample[0] tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]']
# specials & length:
  CLS at: 0 / SEP at: 8 / PAD positions: [9]
  effective (non-PAD) length = 9  / total length = 10
# decode (skip specials): 明日 の 天気 は 晴れ だ 。

# sample[1] tokens:
['[CLS]', 'パソコン', 'が', '急', 'に', '動か', 'なく', 'なっ', 'た', '[SEP]']
# specials & length:
  CLS at: 0 / SEP at: 9 / PAD positions: []
  effective (non-PAD) length = 10  / total length = 10
# decode (skip specials): パソコン が 急 に 動か なく なっ た

# sample[0] raw_subwords=7 capacity=8 -> truncated(est)=0

# sample[1] raw_subwords=9 capacity=8 -> truncat

In [12]:
# 4-12
# =========================================================
# 理論メモ：padding='longest'（動的パディング）の意味と利点
# ---------------------------------------------------------
# ■ 目的
#  - 同一ミニバッチ内で「最長系列（＝各サンプルのトークン長＋[CLS]/[SEP]）」に合わせて右詰めPad。
#  - 各サンプルの「足りないぶん」だけ [PAD] を付与し、attention_mask=0 で無視させる。
#
# ■ 利点（O(L^2 H)観点）
#  - BERTのSelf-Attention計算量は O(L^2 H)。固定長（max_length）で過剰Padを入れると無駄が増える。
#  - 動的パディングは **バッチごとに最小限の長さ**に抑えるため、計算・メモリ効率が向上。
#
# ■ 注意
#  - 右側切り詰め（truncation）はここでは指定しない（=元の長さを保つ）。長文が来るとそのまま長くなる。
#  - 単文入力の token_type_ids は通常ゼロ列（文対時に A=0, B=1）。
#  - WWM（Whole Word Masking）は事前学習時のマスク戦略であり、ここでのパディング挙動自体は変わらない。
# =========================================================

# 保険：tokenizer 未定義なら初期化（4-3 相当）
try:
    tokenizer
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# 保険：text_list 未定義なら用意（4-11 と同一想定）
try:
    text_list
except NameError:
    text_list = ["明日の天気は晴れだ。", "パソコンが急に動かなくなった。"]

# --- エンコード：動的パディング（最長列に合わせる）---
encoding = tokenizer(
    text_list,
    padding="longest",  # ← バッチ内の最長列に合わせてPad
    # truncation=False  # ← 既定（バッチ内では切り詰めない）
)
print("# keys:", list(encoding.keys()))
print("# input_ids[0]:", encoding["input_ids"][0])
print("# input_ids[1]:", encoding["input_ids"][1])
print("# attention_mask[0]:", encoding["attention_mask"][0])
print("# attention_mask[1]:", encoding["attention_mask"][1])
print("# token_type_ids[0]:", encoding["token_type_ids"][0])
print("# token_type_ids[1]:", encoding["token_type_ids"][1])

# --- 可視化：各サンプルのトークン列（特殊トークン込み）---
for i, ids in enumerate(encoding["input_ids"]):
    toks = tokenizer.convert_ids_to_tokens(ids)
    print(f"\n# sample[{i}] tokens:")
    print(toks)

# --- 理論チェック：有効長（= attention_mask の合計）が「最長列」に合っているか ---
PAD_ID = tokenizer.pad_token_id
eff_lens = []
for i, (ids, mask) in enumerate(zip(encoding["input_ids"], encoding["attention_mask"])):
    eff_len = sum(mask)  # 非PAD長
    pad_pos = [j for j, t in enumerate(ids) if t == PAD_ID]
    eff_lens.append(eff_len)
    print(
        f"# sample[{i}] effective_len={eff_len} / total={len(ids)} / PAD positions={pad_pos}"
    )
print("# longest effective_len in batch =", max(eff_lens))

# --- 実運用（学習/推論向け）：PyTorchテンソルでの出力 ---
batch_pt = tokenizer(
    text_list,
    padding="longest",
    return_tensors="pt",  # ← B×L のテンソル（動的にLが決まる）
)
print(
    "\n# tensor shapes:",
    "input_ids",
    tuple(batch_pt["input_ids"].shape),
    "attention_mask",
    tuple(batch_pt["attention_mask"].shape),
    "token_type_ids",
    tuple(batch_pt["token_type_ids"].shape),
)

# 参考：固定長（max_length）との比較（過剰Padを可視化）
batch_fixed = tokenizer(
    text_list,
    max_length=24,  # 仮に固定長を大きめに設定
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)
print(
    "\n# fixed-length tensor shapes:",
    "input_ids",
    tuple(batch_fixed["input_ids"].shape),
    "attention_mask",
    tuple(batch_fixed["attention_mask"].shape),
)
print("# fixed non-PAD per sample:", batch_fixed["attention_mask"].sum(dim=1).tolist())
# 理論補足：
# - longest は「そのバッチの最長」に合わせるため、バッチが変われば L も変わる（=動的）。
# - max_length は常に同じ L に張り付ける（=静的）。スループット安定の代わりに無駄Padが乗りやすい。

# keys: ['input_ids', 'token_type_ids', 'attention_mask']
# input_ids[0]: [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0]
# input_ids[1]: [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 8, 3]
# attention_mask[0]: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
# attention_mask[1]: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# token_type_ids[0]: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# token_type_ids[1]: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# sample[0] tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]']

# sample[1] tokens:
['[CLS]', 'パソコン', 'が', '急', 'に', '動か', 'なく', 'なっ', 'た', '。', '[SEP]']
# sample[0] effective_len=9 / total=11 / PAD positions=[9, 10]
# sample[1] effective_len=11 / total=11 / PAD positions=[]
# longest effective_len in batch = 11

# tensor shapes: input_ids (2, 11) attention_mask (2, 11) token_type_ids (2, 11)

# fixed-length tensor shapes: input_ids (2, 24) attention_mask (2, 24)
# fixed non-PAD per sample: [9, 11]


In [13]:
# 4-13
# =========================================================
# 理論メモ：PyTorchテンソル（B×L）でのバッチ前処理と不変量
# ---------------------------------------------------------
# ■ 目的
#   - 下流タスクにそのまま渡せるテンソル（`input_ids`, `attention_mask`, `token_type_ids`）を
#     B×L 形状で得る。ここでは L=10（固定長）・右側切り詰め＋右詰めPad。
#
# ■ 生成物と役割
#   - input_ids      : 語彙ID列（[CLS]/[SEP]/[PAD]を含む）  → 形状 [B, L]
#   - attention_mask : 非PAD=1, PAD=0。Self-Attention の softmax(QK^T/√d_k) 前に
#                     PAD位置へ -∞ 相当を加算し注意を抑制（勾配も基本流れない） → 形状 [B, L]
#   - token_type_ids : 文A=0, 文B=1。単文は通常ゼロ列（NSP/QAなどの文対で利用） → 形状 [B, L]
#
# ■ 切り詰めと特殊トークン配置（単文）
#   - truncation=True → 右側（末尾側）から削る（first sequence を短縮）。
#   - 不変量：index=0 に [CLS]，index=(有効長-1) に [SEP] が来る（有効長 = attention_mask の合計）。
#
# ■ 実務設計（計算量 O(L^2 H) への配慮）
#   - 固定長（max_length）でスループットは安定。ただし過剰Padで無駄計算が増えやすい。
#   - 動的パディング（padding='longest'）は各バッチの最長に合わせるため計算効率が良い。
# =========================================================

# 保険：tokenizer 未定義なら初期化（WWM版日本語BERTの語彙と対応）
try:
    tokenizer
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# 保険：text_list 未定義なら用意（単文のミニバッチ）
try:
    text_list
except NameError:
    text_list = ["明日の天気は晴れだ。", "パソコンが急に動かなくなった。"]

# --- エンコード：B×L テンソルを直接得る（実務の標準パス） ---
import torch

batch_pt = tokenizer(
    text_list,
    max_length=10,  # L=10 に固定
    padding="max_length",  # 右詰めPadで長さを揃える
    truncation=True,  # 右側切り詰め（単文）
    return_tensors="pt",  # ← PyTorch テンソル（B×L）で返す
)

# --- 出力の基本確認 ---
print("# keys:", list(batch_pt.keys()))
for k, v in batch_pt.items():
    print(f"# {k}: shape={tuple(v.shape)}, dtype={v.dtype}, device={v.device}")

# --- 不変量チェック（理論に基づく簡易検証） ---
CLS_ID = tokenizer.cls_token_id
SEP_ID = tokenizer.sep_token_id
PAD_ID = tokenizer.pad_token_id


def check_invariants(i: int):
    ids = batch_pt["input_ids"][i].tolist()
    mask = batch_pt["attention_mask"][i].tolist()
    toks = tokenizer.convert_ids_to_tokens(ids)

    eff_len = sum(mask)  # 非PAD長（=有効長）
    cls_pos = ids.index(CLS_ID) if CLS_ID in ids else None
    sep_pos = ids.index(SEP_ID) if SEP_ID in ids else None

    print(f"\n# sample[{i}]")
    print("tokens:", toks)
    print("effective_len(non-PAD)=", eff_len, "/ total L=", len(ids))
    print("CLS at", cls_pos, "/ SEP at", sep_pos)
    print("PAD positions:", [j for j, t in enumerate(ids) if t == PAD_ID])

    # 理論的不変量の検証（単文前提）
    assert cls_pos == 0, "CLS は index=0 のはず"
    assert sep_pos == eff_len - 1, "SEP は有効長-1 のはず（単文）"
    # 単文なら token_type_ids は全0
    assert torch.all(
        batch_pt["token_type_ids"][i].eq(0)
    ), "単文の token_type_ids は 0 のはず"


for i in range(len(text_list)):
    check_invariants(i)

# --- 可読表示（人間向け）。可逆ではない点に注意（空白/正規化/未知語の影響）。---
for i in range(len(text_list)):
    s = tokenizer.decode(batch_pt["input_ids"][i], skip_special_tokens=True)
    print(f"\n# decode(sample[{i}]):", s)

# 参考：
# - 下流（分類/トークン分類/QA）では、この batch_pt をそのまま model(**batch_pt) に渡す。
# - 勾配は attention_mask=1 の位置から流れる。PAD=0 の位置は Self-Attention で抑制されるため学習に寄与しない。

# keys: ['input_ids', 'token_type_ids', 'attention_mask']
# input_ids: shape=(2, 10), dtype=torch.int64, device=cpu
# token_type_ids: shape=(2, 10), dtype=torch.int64, device=cpu
# attention_mask: shape=(2, 10), dtype=torch.int64, device=cpu

# sample[0]
tokens: ['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]']
effective_len(non-PAD)= 9 / total L= 10
CLS at 0 / SEP at 8
PAD positions: [9]

# sample[1]
tokens: ['[CLS]', 'パソコン', 'が', '急', 'に', '動か', 'なく', 'なっ', 'た', '[SEP]']
effective_len(non-PAD)= 10 / total L= 10
CLS at 0 / SEP at 9
PAD positions: []

# decode(sample[0]): 明日 の 天気 は 晴れ だ 。

# decode(sample[1]): パソコン が 急 に 動か なく なっ た


In [14]:
# 4-14（Mac対応版）
# =========================================================
# 理論メモ：MacBookにおけるPyTorchデバイス選択
# ---------------------------------------------------------
# ■ 背景
#  - macOS（Apple Silicon搭載のMacBook）では CUDA は利用できない。
#  - 代替として PyTorch は Metal（Apple製GPUバックエンド）＝ MPS を提供。
#    → `torch.backends.mps.is_available()` が True なら GPU 相当の計算を MPS で実行可能。
#  - Intel Mac などで MPS が使えない場合は CPU フォールバックが必要。
#
# ■ デバイス選択の方針
#   1) MPS が使えれば 'mps' を使用（Apple SiliconのGPUを活用）
#   2) それ以外で CUDA が使える環境なら 'cuda'（eGPU や別環境への移植時の保険）
#   3) どちらも不可なら 'cpu'
#
# ■ 実務的含意
#  - Self-Attention のアルゴリズム（softmax(QK^T/√d_k) V）はデバイスに依存しない。
#    デバイスは「同じ計算をどこで行うか」を決めるだけで、理論上のモデルは不変。
#  - MPS の半精度（fp16/bfloat16）は演算未対応のオペがある場合があり、基本は fp32 を推奨。
#  - テンソルもモデルも **同じ device** に移動するのが原則（混在はエラー）。
# =========================================================

import torch
from transformers import BertJapaneseTokenizer, BertModel


# --- 1) デバイス検出（Mac優先でMPSを試みる） ---
def get_best_device() -> torch.device:
    # Apple Silicon + macOS 12.3+ で有効
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    # （保険）他環境へ移植されたときのために CUDA も許容
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")


device = get_best_device()
print(f"[info] using device = {device}")

# --- 2) モデルとトークナイザのロード ---
model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# BertModel: 事前学習済みエンコーダ本体（タスク固有ヘッドは無し）
bert = BertModel.from_pretrained(model_name)

# --- 3) モデルをデバイスへ配置 ---
# 以前の `.cuda()` は Mac ではエラー（CUDA非対応）→ `.to(device)` に統一
bert = bert.to(device)
bert.eval()  # 推論時は dropout 等を停止

# --- 4) サンプル前処理：テンソルも同じ device へ ---
text = "明日は自然言語処理の勉強をしよう。"
batch = tokenizer(
    text,
    return_tensors="pt",
    padding=False,
    truncation=True,
    max_length=128,
)

# テンソルをモデルと同じデバイスへ
batch = {k: v.to(device) for k, v in batch.items()}

# --- 5) 前向き計算（推論） ---
with torch.no_grad():
    outputs = bert(**batch, output_hidden_states=False, output_attentions=False)

# 形状確認：last_hidden_state = [B, L, H], pooler_output = [B, H]
print("last_hidden_state:", tuple(outputs.last_hidden_state.shape))
if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
    print("pooler_output   :", tuple(outputs.pooler_output.shape))

# ---------------------------------------------------------
# 補足（理論と実務のTips）
# - MPSはApple GPU向けバックエンド。計算量は依然 O(L^2 H) のため、mac上でも max_length 設計で
#   速度・メモリを管理する（長文タスクではスライディングや動的パディングが有効）。
# - 学習時：mixed precisionはMPSで未対応opがある場合があるため、まずは fp32 で安定化→段階的に検討。
# - 乱数再現：`torch.manual_seed(...)` を設定し、バージョン・辞書・語彙を固定（日本語は形態素辞書差に注意）。
# ---------------------------------------------------------

[info] using device = mps


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

last_hidden_state: (1, 13, 768)
pooler_output   : (1, 768)


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [15]:
# 4-16（Mac対応・理論コメント付き）
# =========================================================
# 目的：
#  - 2文を日本語BERT（WWM版）へ入力し、最終層の隠れ状態（last_hidden_state: [B, L, H]）を取得する。
#  - MacBook（Apple Silicon想定）では CUDA ではなく Metal(MPS) を優先的に用い、テンソル/モデルを同一デバイスへ移動。
#
# 理論メモ（重要ポイント）：
#  1) 前処理（tokenizer）
#     - 日本語は「形態素解析 → WordPiece」の二段でサブワード化される。
#       同じ文でも辞書・語彙・バージョンで分割境界が揺れる（＝ID列は固定ではない）。
#     - 特殊トークン：[CLS]（先頭）, [SEP]（文末／区切り）。バッチ整列では [PAD] が追加される。
#     - attention_mask=0 の位置（PAD）は Self-Attention の softmax 前に -∞ 相当を加算し注意を抑制 → 学習・推論に寄与しない。
#
#  2) BERTエンコーダ（BertModel）
#     - 出力：last_hidden_state[B,L,H]（各トークンの文脈化表現）、pooler_output[B,H]（[CLS]に線形+Tanh）
#     - Self-Attention：softmax(QK^T / √d_k) V。系列長 L に対して計算量 O(L^2 H)、メモリも概ね O(L^2)。
#       → max_length の設計・パディング方針（'longest' 動的パディング等）がスループットとメモリに直結。
#
#  3) Macでのデバイス選択
#     - Apple Silicon + macOS では CUDA は使えない → torch.backends.mps（Metal）が True なら 'mps' を使用。
#     - モデルとテンソルは必ず同じ device に乗せる（不一致は RuntimeError の主要因）。
#
#  4) テキスト差分とトークナイズ
#     - 「マシーンラーニング」 vs 「マシンラーニング」の表記差は形態素・サブワード境界に影響し得る。
#       WWM（Whole Word Masking）は“事前学習時の語単位マスク戦略”であり、推論時の分割表示自体は変わらない。
# =========================================================

from typing import Dict
import torch
from transformers import BertJapaneseTokenizer, BertModel


# --- デバイス検出（Mac優先：MPS → CUDA → CPU） ---
def get_best_device() -> torch.device:
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")  # Apple GPU (Metal)
    if torch.cuda.is_available():
        return torch.device("cuda")  # 他環境へ移植時の保険
    return torch.device("cpu")


device = get_best_device()
print(f"[info] device = {device}")

# --- モデル & トークナイザ（WWM版とペアで統一） ---
model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
try:
    tokenizer  # 既存セッションにあれば再利用
except NameError:
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

try:
    bert  # 既存セッションにあれば再利用
    # 既存モデルが別deviceなら移し替える
    bert = bert.to(device)
except NameError:
    bert = BertModel.from_pretrained(model_name).to(device)
bert.eval()  # 推論モード（dropout停止）; 学習なら train() に切替

# --- 入力文（B=2）：表記差によりサブワード分割が変わる可能性に留意 ---
text_list = [
    "明日は自然言語処理の勉強をしよう。",
    "明日はマシーンラーニングの勉強をしよう。",
]

# --- 文章の符号化：固定長（L=32）; PADはattention_mask=0で無視される ---
# 返り値（return_tensors='pt'）：
#   input_ids[B,L], attention_mask[B,L], token_type_ids[B,L] （単文は原則0列）
encoding: Dict[str, torch.Tensor] = tokenizer(
    text_list, max_length=32, padding="max_length", truncation=True, return_tensors="pt"
)

# --- データをモデルと同じデバイスへ（Macでは 'mps' が想定；.cuda() は使用しない） ---
encoding = {k: v.to(device) for k, v in encoding.items()}

# --- 前向き計算（推論）。学習で勾配が要らなければ no_grad で省メモリ・高速化 ---
with torch.no_grad():
    # 出力：BaseModelOutputWithPoolingAndCrossAttentions
    output = bert(**encoding, output_hidden_states=False, output_attentions=False)

# --- 最終層の出力（各トークンの文脈化表現）：形状 [B, L, H] ---
last_hidden_state: torch.Tensor = output.last_hidden_state

# --- 参考：shapeと不変量（単文）を軽く確認 ---
B, L, H = last_hidden_state.shape
cls_id = tokenizer.cls_token_id
sep_id = tokenizer.sep_token_id
pad_id = tokenizer.pad_token_id

print(
    f"[info] last_hidden_state shape = {last_hidden_state.shape}"
)  # 例：torch.Size([2, 32, 768])
for i in range(B):
    ids = encoding["input_ids"][i].tolist()
    mask = encoding["attention_mask"][i].tolist()
    eff_len = sum(mask)  # 非PAD長（=有効長）
    cls_pos = ids.index(cls_id) if cls_id in ids else None
    sep_pos = ids.index(sep_id) if sep_id in ids else None
    print(f"[sample {i}] effective_len={eff_len}, CLS@{cls_pos}, SEP@{sep_pos}")

    # 不変量チェック（単文前提）：CLSは先頭、SEPは有効長-1
    assert cls_pos == 0, "CLS は index=0 のはず（単文）"
    assert sep_pos == eff_len - 1, "SEP は有効長-1 のはず（単文）"

# --- 補足：下流タスク接続例 ---
# 文分類：vec = output.pooler_output もしくは last_hidden_state[:, 0, :] を線形層へ
# トークン分類：last_hidden_state を各位置で線形→softmax（PAD位置は mask で損失除外 or ignore_index）
# QA：質問・文脈ペア（[CLS] Q [SEP] C [SEP]）で start/end ロジットを回帰（長文はstride+overlapで分割）

[info] device = mps
[info] last_hidden_state shape = torch.Size([2, 32, 768])
[sample 0] effective_len=13, CLS@0, SEP@12
[sample 1] effective_len=14, CLS@0, SEP@13


In [16]:
# 4-17（Mac対応・理論コメント込み）
# =========================================================
# 目的：
#  - BERTエンコーダ（BertModel）へ「input_ids / attention_mask / token_type_ids」を明示的に渡し、
#    出力（last_hidden_state, pooler_output など）を取得する。
#
# 理論メモ（引数の意味と学習への影響）：
#  - input_ids        : サブワードの語彙ID列（[CLS]/[SEP]/[PAD] 含む）。形状 [B, L]
#  - attention_mask   : 非PAD=1, PAD=0。Self-Attention の softmax(QK^T/√d_k) 前に
#                       PAD位置へ -∞ 相当を加算して注意を抑制 → PADは表現学習と勾配から実質除外。
#  - token_type_ids   : 文A=0, 文B=1。単文は原則0列（NSP/QAなどの文対で区別）。
#  - 出力:
#     * last_hidden_state [B, L, H]：各トークン位置の文脈化表現
#     * pooler_output    [B, H]    ：[CLS]に線形＋tanh（文分類の初期ベースライン）
#  - 計算量：Self-Attention は O(L^2 H)。max_length とパディング設計（'longest'動的パディング等）が
#    速度・メモリに直結する。デバイス（CPU/MPS/CUDA）は計算場所の違いであり理論は不変。
# =========================================================

from typing import Dict, List
import torch
from transformers import BertJapaneseTokenizer, BertModel


# --- デバイス（Mac優先：MPS → CUDA → CPU） ---
def get_best_device() -> torch.device:
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")  # Apple GPU (Metal)
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")


device = get_best_device()
print(f"[info] device = {device}")

# --- モデル/トークナイザの用意（既存が無ければ初期化） ---
model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
try:
    tokenizer
except NameError:
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

try:
    bert
    bert = bert.to(device)
except NameError:
    bert = BertModel.from_pretrained(model_name).to(device)
bert.eval()  # 推論モード（学習時は train()）

# --- エンコーディング準備（前セルの encoding が無ければ作る） ---
try:
    encoding  # 4-16由来の BatchEncoding / dict[str, Tensor] を想定
except NameError:
    text_list: List[str] = [
        "明日は自然言語処理の勉強をしよう。",
        "明日はマシーンラーニングの勉強をしよう。",
    ]
    encoding = tokenizer(
        text_list,
        max_length=32,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

# テンソルをモデルと同じデバイスへ（Macでは 'mps' が想定；.cuda() は使わない）
encoding = {k: v.to(device) for k, v in encoding.items()}

# --- 前向き計算（勾配不要なら no_grad で省メモリ・高速化） ---
with torch.no_grad():
    output = bert(
        input_ids=encoding["input_ids"],
        attention_mask=encoding["attention_mask"],
        token_type_ids=encoding["token_type_ids"],
    )  # BaseModelOutputWithPoolingAndCrossAttentions

# --- 出力の取り出し ---
last_hidden_state: torch.Tensor = output.last_hidden_state  # [B, L, H]
pooler_output: torch.Tensor = (
    output.pooler_output
)  # [B, H]（存在しない設定もある点に注意）

print("last_hidden_state shape:", tuple(last_hidden_state.shape))
print(
    "pooler_output    shape:",
    tuple(pooler_output.shape) if pooler_output is not None else None,
)

# --- 不変量の軽い検証（単文想定）：CLSは index=0、SEP は有効長-1 ---
CLS_ID = tokenizer.cls_token_id
SEP_ID = tokenizer.sep_token_id
PAD_ID = tokenizer.pad_token_id

for i in range(encoding["input_ids"].size(0)):
    ids = encoding["input_ids"][i].tolist()
    mask = encoding["attention_mask"][i].tolist()
    eff_len = sum(mask)
    cls_pos = ids.index(CLS_ID) if CLS_ID in ids else None
    sep_pos = ids.index(SEP_ID) if SEP_ID in ids else None
    print(f"[sample {i}] effective_len={eff_len}, CLS@{cls_pos}, SEP@{sep_pos}")
    assert cls_pos == 0, "CLS は index=0 のはず（単文）"
    assert sep_pos == eff_len - 1, "SEP は有効長-1 のはず（単文）"

# --- 参考：下流タスク接続（理論観点） ---
# 文分類：vec = pooler_output（または last_hidden_state[:,0,:]）→ Linear(num_labels)
# トークン分類：last_hidden_state を位置ごとに Linear(num_tags) → CrossEntropy（PADは ignore_index または mask で除外）
# QA：ペア入力（[CLS] Q [SEP] C [SEP]）で start/end ロジットを回帰。長文は stride+overlap で分割・統合。

[info] device = mps
last_hidden_state shape: (2, 32, 768)
pooler_output    shape: (2, 768)
[sample 0] effective_len=13, CLS@0, SEP@12
[sample 1] effective_len=14, CLS@0, SEP@13


In [17]:
# 4-18
print(last_hidden_state.size())  # テンソルのサイズ

torch.Size([2, 32, 768])


In [19]:
# エラー原因と修正案（AssertionError: CLS は index=0 のはず）
# =========================================================
# ■ なぜ失敗したか
# - チェック用の ID を `bert.config.cls_token_id / sep_token_id` から読んでいましたが、
#   これは None のことがあり（モデル設定に未保存）、あるいは英語BERTの既定値（101/102）と
#   食い違う可能性があります。
# - 東北大日本語BERTでは一般に [CLS]=2, [SEP]=3, [PAD]=0 です。よって tokenizer から ID を取得すべきです。
#
# ■ 対処方針
# - `tokenizer.cls_token_id / sep_token_id / pad_token_id` を使う。
# - ついでにトークン列で直接確認する安全版（文字列 "[CLS]"/"[SEP]" を探す）も示します。
# - もし `encoding` を作る際に `add_special_tokens=False` を明示していた場合、[CLS]/[SEP] は付きません。
#   （通常は既定 True なので付いています）

# --- 修正済みの不変量チェック（単文前提） ---
CLS_ID = tokenizer.cls_token_id
SEP_ID = tokenizer.sep_token_id
PAD_ID = tokenizer.pad_token_id
print(
    f"[ids] CLS={CLS_ID}, SEP={SEP_ID}, PAD={PAD_ID}, CLS_tok={tokenizer.cls_token}, SEP_tok={tokenizer.sep_token}"
)

for i in range(encoding["input_ids"].size(0)):
    ids = encoding["input_ids"][i].tolist()
    mask = encoding["attention_mask"][i].tolist()
    toks = tokenizer.convert_ids_to_tokens(ids)

    eff_len = sum(mask)  # 非PAD長（=有効長）
    cls_pos = ids.index(CLS_ID) if CLS_ID in ids else None
    sep_pos = ids.index(SEP_ID) if SEP_ID in ids else None

    print(f"[sample {i}] eff_len={eff_len}, CLS@{cls_pos}, SEP@{sep_pos}")
    print("tokens:", toks)

    # ① IDベースの不変量（最速）
    assert cls_pos == 0, "CLS は index=0（先頭）のはず（単文）"
    assert sep_pos == eff_len - 1, "SEP は有効長-1（末尾）のはず（単文）"

    # ② トークン文字列ベースの保険（より頑健）
    #    ※ tokenizer の特別トークン文字列を使って位置を検出
    try:
        cls_pos_tok = toks.index(tokenizer.cls_token)
        sep_pos_tok = toks.index(tokenizer.sep_token)
        assert cls_pos_tok == 0, "トークン '[CLS]' は先頭のはず（単文）"
        assert sep_pos_tok == eff_len - 1, "トークン '[SEP]' は有効長-1 のはず（単文）"
    except ValueError:
        # [CLS]/[SEP] が見つからない場合は、add_special_tokens=False で符号化していないか確認する
        raise AssertionError(
            "特殊トークンが見つかりません。encoding を作る際に add_special_tokens=True（既定）を使っているか確認してください。"
        )

# --- 参考：encoding を作る際の推奨（既定で特殊トークン付与されます） ---
# encoding = tokenizer(
#     text_list,
#     max_length=32,
#     padding='max_length',
#     truncation=True,
#     return_tensors='pt'     # add_special_tokens は既定 True（[CLS]/[SEP] 付与）
# )
# encoding = {k: v.to(device) for k, v in encoding.items()}

# --- 注意：ペア入力（質問+文脈など）の場合 ---
# [CLS] A [SEP] B [SEP] の形になるため、SEP は2つ現れる。単文用の不変量アサートはそのまま使えません。
# その場合は token_type_ids（A=0, B=1）で区間を分けて確認してください。

[ids] CLS=2, SEP=3, PAD=0, CLS_tok=[CLS], SEP_tok=[SEP]
[sample 0] eff_len=13, CLS@0, SEP@12
tokens: ['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', '##う', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[sample 1] eff_len=14, CLS@0, SEP@13
tokens: ['[CLS]', '明日', 'は', 'マ', '##シーン', 'ラー', '##ニング', 'の', '勉強', 'を', 'しよ', '##う', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [20]:
# 4-20（テンソル→CPU→NumPy→Pythonリストへの変換：理論コメント付き）
# =========================================================
# 目的：
#  - BERTの出力 last_hidden_state（形状 [B, L, H] の torch.Tensor）を
#    推論後の解析やシリアライズ（JSON, CSVなど）で扱いやすい形式に落とす。
#
# 理論メモ：
#  - PyTorch Tensor → NumPy は **同一メモリを共有**（コピー無し）が基本だが、
#    共有できるのは **CPU上の連続（contiguous）浮動小数テンソル**のみ。
#    GPU(CUDA/MPS) 上の Tensor は **いったん .to('cpu')** でホスト側へ移す必要がある。
#  - 推論時でもグラフが付いた Tensor を直接 .numpy() するのは非推奨。
#    **.detach()** で計算グラフから切り離してから変換するのが安全。
#  - リスト化（.tolist()）は **可搬性は高いが非常にメモリと時間を消費**する。
#    大規模行列はできる限り NumPy のまま保存（.npy / .npz）や圧縮型（float16等）を検討。
#
# 実装（安全版の手順）：
#  1) detach：勾配追跡を外す（推論なら原則不要だが安全策）
#  2) to('cpu')：GPU/MPS から CPU へ移動（NumPyはCPU配列のみ）
#  3) contiguous：必要に応じて連続化（ストライドが特殊な場合の保険）
#  4) numpy()：NumPy ndarray へ変換
#  5) tolist()：最終的にPythonリストへ（本当に必要な場合のみ）
# =========================================================

import numpy as np
import math

# （前提）last_hidden_state: torch.Tensor [B, L, H]
# 例：torch.Size([2, 32, 768]) など
print(
    "last_hidden_state (torch) shape:",
    tuple(last_hidden_state.shape),
    "dtype:",
    last_hidden_state.dtype,
    "device:",
    last_hidden_state.device,
)

# --- 推奨：安全な変換パス ---
lhs_cpu = last_hidden_state.detach().to("cpu")  # 1) detach + 2) CPU へ移動
lhs_cpu = lhs_cpu.contiguous()  # 3) 必要に応じて連続化（保険）
lhs_np = lhs_cpu.numpy()  # 4) NumPy ndarray へ（ゼロコピー想定）
print("last_hidden_state (numpy) shape:", lhs_np.shape, "dtype:", lhs_np.dtype)

# --- メモリ見積り（実務上の注意喚起） ---
# bytes = B * L * H * sizeof(dtype)。float32なら 4 バイト
bytes_total = lhs_np.nbytes
print(f"numpy nbytes: {bytes_total:,} bytes ≈ {bytes_total/1024/1024:.2f} MiB")

# --- 省メモリオプション（必要なときのみ） ---
# ※ 数値誤差を許容できるなら、float16 / bfloat16 などへ圧縮して保存する。
# lhs_np16 = lhs_np.astype(np.float16)  # 約半分のサイズ
# np.save('last_hidden_state_fp16.npy', lhs_np16)

# --- JSON等に出すなど「純Python」形式が必要な場合のみリスト化 ---
#    （巨大行列での tolist() は非常に重いので、本当に必要なときに限定）
lhs_list = lhs_np.tolist()
print(
    "converted to Python list (length, depth):",
    len(lhs_list),
    len(lhs_list[0]) if len(lhs_list) > 0 else 0,
)

# --- 参考：よくある落とし穴と対処 ---
# 1) GPU/MPS テンソルで .numpy() → エラー：必ず .to('cpu') してから
# 2) 勾配付きテンソルで .numpy() → 警告/エラー：.detach() を挟む
# 3) 速度/メモリ問題 → できるだけ NumPy のまま保存（.npy/.npz）や圧縮（float16）を検討
# 4) JSONシリアライズ → リスト化は可読だが巨大。集約（平均/プーリング）して次元を落としてからの出力が実務的

last_hidden_state (torch) shape: (2, 32, 768) dtype: torch.float32 device: mps:0
last_hidden_state (numpy) shape: (2, 32, 768) dtype: float32
numpy nbytes: 196,608 bytes ≈ 0.19 MiB
converted to Python list (length, depth): 2 32
